# Predict Podcast Listening Time

## Keras - 1 Attempt

## Attempt 1

In [4]:

# Set seeds for reproducibility
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import time

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Data loading
train_df = pd.read_csv('playground-series-s5e4/train.csv')
test_df = pd.read_csv('playground-series-s5e4/test.csv')

id_col = 'id'
target_columns = ['Listening_Time_minutes']
competition_problem_subtype = 'continuous-regression'

# Prepare target
Y = train_df[target_columns].astype(float).values
if np.all(Y >= 0):
    y_enc = np.log1p(Y)
else:
    y_enc = Y

X = train_df.drop(columns=target_columns + [id_col], errors='ignore')

# Assign train and validation sets (test split provided)
X_train = X.copy()
y_train = y_enc
train_ids = train_df[id_col]

X_val = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering: drop all-missing columns
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val.drop(columns=all_missing, inplace=True)

# Identify categorical columns and drop high-cardinality ones
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
high_card = [c for c in cat_cols if X_train[c].nunique() > 50]
X_train.drop(columns=high_card, inplace=True)
X_val.drop(columns=high_card, inplace=True)
cat_cols = [c for c in cat_cols if c not in high_card]
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture selection
dataset_size, feature_count = X_train_proc.shape
if dataset_size < 10000 or feature_count < 100:
    layer_sizes = [min(feature_count * 2, 128), min(feature_count, 64)]
    dropout_rate = 0.3
    use_batchnorm = False
else:
    raw_sizes = [feature_count * i for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [int(s) for s in raw_sizes if s >= 16]
    dropout_rate = 0.4
    use_batchnorm = True

model = Sequential()
for idx, size in enumerate(layer_sizes):
    if idx == 0:
        model.add(Dense(size, activation='relu', input_shape=(feature_count,)))
    else:
        model.add(Dense(size, activation='relu'))
    model.add(Dropout(dropout_rate))
    if use_batchnorm:
        from tensorflow.keras.layers import BatchNormalization
        model.add(BatchNormalization())

# Output layer for regression
model.add(Dense(1, activation='linear'))

# Compile
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[RootMeanSquaredError(), MeanAbsoluteError()]
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=2
)
training_duration = time.time() - start_time

# Logging results
results = {
    'training_accuracy': None,
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': None,
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction on test set
raw_preds = model.predict(X_val_proc)
final_preds = raw_preds
if np.all(final_preds >= 0):
    final_preds = np.expm1(np.clip(final_preds, a_min=None, a_max=20))
final_preds = final_preds.reshape(-1, 1)

# Prepare submission
submission = pd.DataFrame(final_preds, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)


/home/exouser/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100

Epoch 1: val_loss improved from inf to 0.18842, saving model to best_model.h5


18750/18750 - 28s - 1ms/step - loss: 0.2810 - mean_absolute_error: 0.3736 - root_mean_squared_error: 0.5301 - val_loss: 0.1884 - val_mean_absolute_error: 0.2763 - val_root_mean_squared_error: 0.4341
Epoch 2/100

Epoch 2: val_loss did not improve from 0.18842
18750/18750 - 28s - 1ms/step - loss: 0.1925 - mean_absolute_error: 0.2972 - root_mean_squared_error: 0.4388 - val_loss: 0.1917 - val_mean_absolute_error: 0.2767 - val_root_mean_squared_error: 0.4379
Epoch 3/100

Epoch 3: val_loss improved from 0.18842 to 0.18401, saving model to best_model.h5


18750/18750 - 25s - 1ms/step - loss: 0.1892 - mean_absolute_error: 0.2933 - root_mean_squared_error: 0.4350 - val_loss: 0.1840 - val_mean_absolute_error: 0.2742 - val_root_mean_squared_error: 0.4290
Epoch 4/100

Epoch 4: val_loss improved from 0.18401 to 0.18187, saving model to best_model.h5


18750/18750 - 26s - 1ms/step - loss: 0.1874 - mean_absolute_error: 0.2919 - root_mean_squared_error: 0.4329 - val_loss: 0.1819 - val_mean_absolute_error: 0.2737 - val_root_mean_squared_error: 0.4265
Epoch 5/100

Epoch 5: val_loss improved from 0.18187 to 0.17826, saving model to best_model.h5


...

18750/18750 - 32s - 2ms/step - loss: 0.1835 - mean_absolute_error: 0.2891 - root_mean_squared_error: 0.4284 - val_loss: 0.1742 - val_mean_absolute_error: 0.2748 - val_root_mean_squared_error: 0.4174
Epoch 16/100

Epoch 16: val_loss improved from 0.17422 to 0.17187, saving model to best_model.h5


18750/18750 - 28s - 2ms/step - loss: 0.1834 - mean_absolute_error: 0.2892 - root_mean_squared_error: 0.4283 - val_loss: 0.1719 - val_mean_absolute_error: 0.2709 - val_root_mean_squared_error: 0.4146
Epoch 17/100

Epoch 17: val_loss improved from 0.17187 to 0.17163, saving model to best_model.h5


18750/18750 - 28s - 1ms/step - loss: 0.1834 - mean_absolute_error: 0.2891 - root_mean_squared_error: 0.4283 - val_loss: 0.1716 - val_mean_absolute_error: 0.2695 - val_root_mean_squared_error: 0.4143
Epoch 18/100

Epoch 18: val_loss did not improve from 0.17163
18750/18750 - 28s - 2ms/step - loss: 0.1834 - mean_absolute_error: 0.2891 - root_mean_squared_error: 0.4283 - val_loss: 0.1742 - val_mean_absolute_error: 0.2710 - val_root_mean_squared_error: 0.4174
Epoch 19/100

Epoch 19: val_loss did not improve from 0.17163
18750/18750 - 27s - 1ms/step - loss: 0.1827 - mean_absolute_error: 0.2887 - root_mean_squared_error: 0.4274 - val_loss: 0.1728 - val_mean_absolute_error: 0.2722 - val_root_mean_squared_error: 0.4157
Epoch 20/100

Epoch 20: val_loss did not improve from 0.17163
18750/18750 - 28s - 1ms/step - loss: 0.1830 - mean_absolute_error: 0.2889 - root_mean_squared_error: 0.4278 - val_loss: 0.1760 - val_mean_absolute_error: 0.2706 - val_root_mean_squared_error: 0.4195
Epoch 21/100

Epoc

18750/18750 - 28s - 1ms/step - loss: 0.1830 - mean_absolute_error: 0.2890 - root_mean_squared_error: 0.4278 - val_loss: 0.1707 - val_mean_absolute_error: 0.2702 - val_root_mean_squared_error: 0.4131
Epoch 22/100

Epoch 22: val_loss improved from 0.17067 to 0.16936, saving model to best_model.h5


18750/18750 - 28s - 2ms/step - loss: 0.1835 - mean_absolute_error: 0.2892 - root_mean_squared_error: 0.4283 - val_loss: 0.1694 - val_mean_absolute_error: 0.2700 - val_root_mean_squared_error: 0.4115
Epoch 23/100

Epoch 23: val_loss did not improve from 0.16936
18750/18750 - 32s - 2ms/step - loss: 0.1832 - mean_absolute_error: 0.2891 - root_mean_squared_error: 0.4280 - val_loss: 0.1707 - val_mean_absolute_error: 0.2735 - val_root_mean_squared_error: 0.4131
Epoch 24/100

Epoch 24: val_loss did not improve from 0.16936
18750/18750 - 32s - 2ms/step - loss: 0.1829 - mean_absolute_error: 0.2890 - root_mean_squared_error: 0.4277 - val_loss: 0.1724 - val_mean_absolute_error: 0.2715 - val_root_mean_squared_error: 0.4153
Epoch 25/100

Epoch 25: val_loss did not improve from 0.16936
18750/18750 - 27s - 1ms/step - loss: 0.1831 - mean_absolute_error: 0.2891 - root_mean_squared_error: 0.4279 - val_loss: 0.1739 - val_mean_absolute_error: 0.2719 - val_root_mean_squared_error: 0.4171
Epoch 26/100

Epoc

## Keras Tuner - 1 Attempt

## Attempt 1


In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import time

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Data loading
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

id_col = 'id'
target_columns = ['Listening_Time_minutes']
competition_problem_subtype = 'continuous-regression'

# Prepare target
Y = train_df[target_columns].astype(float).values
if np.all(Y >= 0):
    y_enc = np.log1p(Y)
else:
    y_enc = Y

X = train_df.drop(columns=target_columns + [id_col], errors='ignore')

# Assign train and validation sets (test split provided)
X_train = X.copy()
y_train = y_enc
train_ids = train_df[id_col]

X_val = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering: drop all-missing columns
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val.drop(columns=all_missing, inplace=True)

# Identify categorical columns and drop high-cardinality ones
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
high_card = [c for c in cat_cols if X_train[c].nunique() > 50]
X_train.drop(columns=high_card, inplace=True)
X_val.drop(columns=high_card, inplace=True)
cat_cols = [c for c in cat_cols if c not in high_card]
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture selection
dataset_size, feature_count = X_train_proc.shape
if dataset_size < 10000 or feature_count < 100:
    layer_sizes = [min(feature_count * 2, 128), min(feature_count, 64)]
    dropout_rate = 0.3
    use_batchnorm = False
else:
    raw_sizes = [feature_count * i for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [int(s) for s in raw_sizes if s >= 16]
    dropout_rate = 0.4
    use_batchnorm = True

import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = feature_count

# HyperModel
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        model = Sequential()
        for idx in range(layers):
            if idx == 0:
                model.add(Dense(units, activation='relu', input_shape=(n_features,)))
            else:
                model.add(Dense(units, activation='relu'))
            model.add(Dropout(drop))
            model.add(BatchNormalization())
        model.add(Dense(1, activation='linear'))  # Output layer for regression
        model.compile(optimizer=opt, loss='mean_squared_error', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])
        return model

# Tuner
bs = 64  # batch size
ep = 100  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train_proc, y_train,
    validation_split=0.2,
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=2
)
training_duration = time.time() - start_time

# Logging results
results = {
    'training_accuracy': None,
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': None,
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction on test set
raw_preds = model.predict(X_val_proc)
final_preds = raw_preds
if np.all(final_preds >= 0):
    final_preds = np.expm1(np.clip(final_preds, a_min=None, a_max=20))
final_preds = final_preds.reshape(-1, 1)

# Prepare submission
submission = pd.DataFrame(final_preds, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 11m 52s]
val_loss: 0.16736340522766113

Best val_loss So Far: 0.16674523055553436
Total elapsed time: 02h 04m 20s
Epoch 1/100


9375/9375 - 50s - 5ms/step - loss: 0.2956 - mean_absolute_error: 0.3486 - root_mean_squared_error: 0.5437 - val_loss: 0.1814 - val_mean_absolute_error: 0.2766 - val_root_mean_squared_error: 0.4260
Epoch 2/100


9375/9375 - 47s - 5ms/step - loss: 0.2012 - mean_absolute_error: 0.3053 - root_mean_squared_error: 0.4486 - val_loss: 0.1748 - val_mean_absolute_error: 0.2752 - val_root_mean_squared_error: 0.4181
Epoch 3/100
9375/9375 - 44s - 5ms/step - loss: 0.1912 - mean_absolute_error: 0.2984 - root_mean_squared_error: 0.4372 - val_loss: 0.1774 - val_mean_absolute_error: 0.2703 - val_root_mean_squared_error: 0.4212
Epoch 4/100


9375/9375 - 46s - 5ms/step - loss: 0.1866 - mean_absolute_error: 0.2951 - root_mean_squared_error: 0.4319 - val_loss: 0.1718 - val_mean_absolute_error: 0.2727 - val_root_mean_squared_error: 0.4145
Epoch 5/100


9375/9375 - 43s - 5ms/step - loss: 0.1837 - mean_absolute_error: 0.2929 - root_mean_squared_error: 0.4286 - val_loss: 0.1707 - val_mean_absolute_error: 0.2705 - val_root_mean_squared_error: 0.4132
Epoch 6/100


9375/9375 - 47s - 5ms/step - loss: 0.1811 - mean_absolute_error: 0.2909 - root_mean_squared_error: 0.4255 - val_loss: 0.1691 - val_mean_absolute_error: 0.2720 - val_root_mean_squared_error: 0.4112
Epoch 7/100


9375/9375 - 45s - 5ms/step - loss: 0.1787 - mean_absolute_error: 0.2892 - root_mean_squared_error: 0.4228 - val_loss: 0.1663 - val_mean_absolute_error: 0.2681 - val_root_mean_squared_error: 0.4078
Epoch 8/100
9375/9375 - 42s - 4ms/step - loss: 0.1764 - mean_absolute_error: 0.2876 - root_mean_squared_error: 0.4201 - val_loss: 0.1715 - val_mean_absolute_error: 0.2728 - val_root_mean_squared_error: 0.4142
Epoch 9/100
9375/9375 - 43s - 5ms/step - loss: 0.1742 - mean_absolute_error: 0.2861 - root_mean_squared_error: 0.4174 - val_loss: 0.1687 - val_mean_absolute_error: 0.2670 - val_root_mean_squared_error: 0.4107
Epoch 10/100
9375/9375 - 46s - 5ms/step - loss: 0.1725 - mean_absolute_error: 0.2849 - root_mean_squared_error: 0.4153 - val_loss: 0.1699 - val_mean_absolute_error: 0.2670 - val_root_mean_squared_error: 0.4121
Epoch 11/100
9375/9375 - 47s - 5ms/step - loss: 0.1705 - mean_absolute_error: 0.2837 - root_mean_squared_error: 0.4130 - val_loss: 0.1689 - val_mean_absolute_error: 0.2662 - v

18750/18750 - 98s - 5ms/step - loss: 0.1934 - mean_absolute_error: 0.3048 - root_mean_squared_error: 0.4397 - val_loss: 0.1694 - val_mean_absolute_error: 0.2694 - val_root_mean_squared_error: 0.4116
Epoch 2/100

Epoch 2: val_loss improved from 0.16944 to 0.16883, saving model to best_model.h5


18750/18750 - 88s - 5ms/step - loss: 0.1893 - mean_absolute_error: 0.3017 - root_mean_squared_error: 0.4351 - val_loss: 0.1688 - val_mean_absolute_error: 0.2707 - val_root_mean_squared_error: 0.4109
Epoch 3/100

Epoch 3: val_loss did not improve from 0.16883
18750/18750 - 92s - 5ms/step - loss: 0.1866 - mean_absolute_error: 0.2997 - root_mean_squared_error: 0.4320 - val_loss: 0.2797 - val_mean_absolute_error: 0.2796 - val_root_mean_squared_error: 0.5288
Epoch 4/100

Epoch 4: val_loss did not improve from 0.16883
18750/18750 - 80s - 4ms/step - loss: 0.1841 - mean_absolute_error: 0.2977 - root_mean_squared_error: 0.4290 - val_loss: 1.5239 - val_mean_absolute_error: 0.2862 - val_root_mean_squared_error: 1.2344
Epoch 5/100

Epoch 5: val_loss did not improve from 0.16883
18750/18750 - 88s - 5ms/step - loss: 0.1822 - mean_absolute_error: 0.2963 - root_mean_squared_error: 0.4268 - val_loss: 0.1794 - val_mean_absolute_error: 0.2697 - val_root_mean_squared_error: 0.4236
Epoch 6/100

Epoch 6: va

In [ ]:
print(training_duration)

1468.453455
